In [1]:
def readdata(fileName):  
    temp_list = []
    file = open(fileName, 'r', encoding='utf-8')
    Lines = file.readlines()
    for line in Lines:    
            temp_list.append(line.rstrip("\n"))
    return temp_list 

In [2]:
import numpy as np
def get_masked_input_and_labels(source):
    temp_dict1 = {}
    temp_dict2 = {}
    for i, item in enumerate(source):
        temp_dict1[item] = i
    temp_dict1['<mask>'] = len(source)  
    
    for i, item in enumerate(source):
        temp_dict2[i] = item
    temp_dict2[len(source)] = '<mask>' 
    
    mask_token_id= temp_dict1['<mask>']
    encoded_texts = np.array([temp_dict1[item] for item in source])
    # 15% BERT masking
    inp_mask = np.random.rand(len(encoded_texts)) < 0.25
    inp_mask[encoded_texts <= 4] = False
    # Prepare input
    encoded_texts_masked = np.copy(encoded_texts)
    # Set input to [MASK] which is the last token for the 90% of tokens
    # This means leaving 10% unchanged
    inp_mask_2mask = inp_mask & (np.random.rand(*encoded_texts.shape) < 0.80)
    encoded_texts_masked[
        inp_mask_2mask
    ] = mask_token_id  # mask token is the last in the dict

    # Set 10% to a random token
    inp_mask_2random = inp_mask_2mask & (np.random.rand(*encoded_texts.shape) < 2 / 8)
    encoded_texts_masked[inp_mask_2random] = np.random.randint(
        3, mask_token_id, inp_mask_2random.sum()
    )

    encoded_masked_texts = [temp_dict2[item] for item in encoded_texts_masked]
    return  ' '.join(source[4:])+' \n', ' '.join(encoded_masked_texts)+' \n'

In [3]:
isk_data = readdata('tokens.txt')
isk_input_list = []
isk_target_list = []
for data_point in isk_data: 
    sentence = data_point.strip().split()
    non_masked_sent, masked_sent = get_masked_input_and_labels(sentence)
    isk_input_list.append(masked_sent)
    isk_target_list.append(non_masked_sent) 

In [4]:
# Define the split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate the split indices
train_end = int(len(isk_input_list) * train_ratio)
val_end = train_end + int(len(isk_input_list) * val_ratio)

# Write files with isk_input_list and isk_target_list in train, val, and test files
with open('train.isk', 'w', encoding='utf-8') as f:
    for item in isk_input_list[:train_end]:
        f.write("%s" % item)
with open('val.isk', 'w', encoding='utf-8') as f:
    for item in isk_input_list[train_end:val_end]:
        f.write("%s" % item)
with open('test.isk', 'w', encoding='utf-8') as f:
    for item in isk_input_list[val_end:]:
        f.write("%s" % item)

with open('train.isk.target', 'w', encoding='utf-8') as f:
    for item in isk_target_list[:train_end]:
        f.write("%s" % item)
with open('val.isk.target', 'w', encoding='utf-8') as f:
    for item in isk_target_list[train_end:val_end]:
        f.write("%s" % item)
with open('test.isk.target', 'w', encoding='utf-8') as f:
    for item in isk_target_list[val_end:]:
        f.write("%s" % item)

In [ ]:
hindi_data = readdata('English/en_2Mmono_train.spm.hi_IN')
hindi_input_list = []
hindi_target_list = []
for data_point in hindi_data: 
    sentence = data_point.strip().split()
    non_masked_sent, masked_sent = get_masked_input_and_labels(sentence)
    hindi_input_list.append(masked_sent)
    hindi_target_list.append(non_masked_sent) 

In [ ]:
assert len(hindi_input_list) == len(hindi_target_list)    
print(len(hindi_input_list))
print(hindi_input_list[10000])
print(hindi_target_list[10000])

In [ ]:
ja_data = readdata('Japanese/ja_2Mmono_train.spm.hi_IN')
print(len(ja_data))
ja_input_list = []
ja_target_list = []
for data_point in ja_data: 
    sentence = data_point.strip().split()
    non_masked_sent, masked_sent = get_masked_input_and_labels(sentence)
    ja_input_list.append(masked_sent)
    ja_target_list.append(non_masked_sent)

In [ ]:
assert len(ja_input_list) == len(ja_target_list)    
print(len(ja_input_list))
print(ja_input_list[100])
print(ja_target_list[100])

In [198]:
hi_data = readdata('Hindi/hi_2Mmono_train.spm.hi_IN')
hi_input_list = []
hi_target_list = []
for data_point in hi_data: 
    sentence = data_point.strip().split()
    non_masked_sent, masked_sent = get_masked_input_and_labels(sentence)
    hi_input_list.append(masked_sent)
    hi_target_list.append(non_masked_sent)

In [ ]:
assert len(hi_input_list) == len(hi_target_list)    
print(len(hi_input_list))
print(hi_input_list[1000])
print(hi_target_list[1000])

In [208]:
input_list = []
target_list = []
for i in range(len(hi_input_list)*3):
    input_list.append(hindi_input_list)
    input_list.append(ja_input_list)
    input_list.append(hi_input_list)
    
    target_list.append(hindi_target_list)
    target_list.append(ja_target_list)
    target_list.append(hi_target_list)

In [ ]:
assert len(input_list) == len(target_list)    
print(len(input_list))
print(input_list[:6])
print("*"*100)
print(target_list[:6])

In [6]:
file_mapping = {
    'ja_2Mmono_train.hi_IN': final_mono_hindi
}
for k, v in file_mapping.items():
    with open(f'{k}', 'w') as fp:
        fp.writelines(v)